# Feature Engineering

***

## 1. Data Cleansing

In this first part we retrieve the data from some HDF5 file, change some column formats to their relevant data type and drop other columns that aren't needed for the subsequent analysis.

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
# functions definitions

def readHdf5(path):  # use pandas to read data from the hdf5 file
    df = pd.read_hdf(path + 'h5File.h5', 'df')
    return df

def columnsFormat(df): # change date column format to datetime
    df['date'] = pd.to_datetime(df['date'])
    df['cancelled'] = df['cancelled'].astype(bool)
    return df

def dropColumns(df, columns): # drop columns that are not needed for the model 
    df = df.drop(columns, axis=1)
    return df

def toHdf5Clean(df, path): # export the pandas data frame into HDF5
    df.to_hdf(path + 'h5File_clean.h5', key='df', mode='w')

In [3]:
path = '../'
df_file = readHdf5(path)
df_format = columnsFormat(df_file)
columns = ['year','month','day-of-month','day-of-week','actual-elapsed-time','distance','cancelled']
df = dropColumns(df_format, columns)
df.dtypes

date                   datetime64[ns]
origin                         object
destination                    object
departure-time                float64
departure-delay               float64
taxi-out                      float64
taxi-in                       float64
arrival-time                  float64
arrival-delay                 float64
air-time                      float64
carrier-delay                 float64
weather-delay                 float64
nas-delay                     float64
security-delay                float64
late-aircraft-delay           float64
dtype: object

In [4]:
toHdf5Clean(df, path) # save the cleaned data frame

## 2. Feature Engineering

**Feature creation**

Here we create the new columns that we'll need: months (1 to 12), days (1 to 31), days of the week (0 to 6), delayed (true or false) and status (0, 1 or 2, for on time, slightly delayed and highly delayed, respectively).

In [5]:
# function definitions

def dateColumns(df):
    df['month'] = df['date'].dt.month
    df['day-of-the-month'] = df['date'].dt.day
    df['day-of-the-week'] = df['date'].dt.dayofweek
    return df
    
def delayedColumn(df):
    df['delayed'] = df['arrival-delay'].apply(lambda x: True if x >= 15 else False)
    return df
    
def statusColumn(df):
    df['status'] = df['arrival-delay'].apply(lambda x: 0 if x < 15 else (1 if x < 60 else 2))
    return df

In [6]:
df_cols1 = dateColumns(df)
df_cols2 = delayedColumn(df_cols1)
df = statusColumn(df_cols2)
df.head()

,date,origin,destination,departure-time,departure-delay,taxi-out,taxi-in,arrival-time,arrival-delay,air-time,carrier-delay,weather-delay,nas-delay,security-delay,late-aircraft-delay,month,day-of-the-month,day-of-the-week,delayed,status
342111,2019-02-12,IAH,LAS,1941.0,-4.0,23.0,17.0,2133.0,28.0,192.0,0.0,0.0,28.0,0.0,0.0,2,12,1,True,1
298420,2019-07-14,BWI,FLL,2125.0,0.0,9.0,13.0,2346.0,-19.0,119.0,NaN,NaN,NaN,NaN,NaN,7,14,6,False,0
275467,2019-04-27,OMA,ATL,1035.0,-2.0,8.0,3.0,1335.0,-19.0,109.0,NaN,NaN,NaN,NaN,NaN,4,27,5,False,0
271709,2019-06-03,IAH,LAX,1431.0,-6.0,20.0,7.0,1557.0,-8.0,179.0,NaN,NaN,NaN,NaN,NaN,6,3,0,False,0
50814,2019-10-09,MSP,MBS,2020.0,-4.0,11.0,1.0,2236.0,-27.0,64.0,NaN,NaN,NaN,NaN,NaN,10,9,2,False,0


**Feature transformation**

* We checked the numerical columns for missing values using the isna().sum() in the data exploration notebook. Here, we imput them using the linear interpolate function. The following columns had missing values and need to be handled: departure-time, departure-delay, taxi-out, taxi-in, arrival-time, arrival-delay, air-time, carrier-delay, weather-delay, nas-delay, security-delay, late-aircraft-delay.
* Next, we standardize all numerical variables using the preprocessing package from sklearn.
* Finally, two categorical features are encoded.

In [7]:
# function definitions

def nullImputation(df, columns): # null value imputation using interpolation method
    for col in columns:
        df[col] = df[col].interpolate(method='linear', limit_direction ='both', axis=0)
    return df

def standarize(df, columns): # standardize variables
    #num_features = df.select_dtypes(include=[np.number]).columns
    df[columns] = preprocessing.MinMaxScaler().fit_transform(df[columns])
    return df

def encode(df, columns): # encode categorical features
    for col in columns:
        df[col] = df[col].astype('category').cat.codes
    return df

def toHdf5FeatureEng(df, path): # export the pandas data frame into HDF5
    df.to_hdf(path + 'h5File_FE.h5', key='df', mode='w')

In [8]:
columns = ['taxi-in','arrival-time','arrival-delay','air-time','carrier-delay',
           'weather-delay','nas-delay','security-delay','late-aircraft-delay']
df_imputation = nullImputation(df, columns)
df_imputation.isna().sum()

date                   0
origin                 0
destination            0
departure-time         0
departure-delay        0
taxi-out               0
taxi-in                0
arrival-time           0
arrival-delay          0
air-time               0
carrier-delay          0
weather-delay          0
nas-delay              0
security-delay         0
late-aircraft-delay    0
month                  0
day-of-the-month       0
day-of-the-week        0
delayed                0
status                 0
dtype: int64

In [9]:
columns = ['departure-time','departure-delay','taxi-out','taxi-in','arrival-time',
           'arrival-delay','air-time','carrier-delay','weather-delay','nas-delay',
           'security-delay','late-aircraft-delay']
df_standarized = standarize(df_imputation, columns)
df_standarized.head()

,date,origin,destination,departure-time,departure-delay,taxi-out,taxi-in,arrival-time,arrival-delay,air-time,carrier-delay,weather-delay,nas-delay,security-delay,late-aircraft-delay,month,day-of-the-month,day-of-the-week,delayed,status
342111,2019-02-12,IAH,LAS,0.808670,0.016538,0.13750,0.064516,0.888704,0.037459,0.269400,0.0,0.0,0.019608,0.0,0.000000,2,12,1,True,1
298420,2019-07-14,BWI,FLL,0.885369,0.018008,0.05000,0.048387,0.977491,0.020198,0.162518,0.0,0.0,0.016340,0.0,0.003692,7,14,6,False,0
275467,2019-04-27,OMA,ATL,0.431013,0.017273,0.04375,0.008065,0.556065,0.020198,0.147877,0.0,0.0,0.013072,0.0,0.007383,4,27,5,False,0
271709,2019-06-03,IAH,LAX,0.596082,0.015803,0.11875,0.024194,0.648604,0.024238,0.250366,0.0,0.0,0.009804,0.0,0.011075,6,3,0,False,0
50814,2019-10-09,MSP,MBS,0.841601,0.016538,0.06250,0.000000,0.931638,0.017260,0.081991,0.0,0.0,0.006536,0.0,0.014766,10,9,2,False,0


In [10]:
columns = ['origin','destination']
df_encoded = encode(df_standarized, columns)
df_encoded.head()

,date,origin,destination,departure-time,departure-delay,taxi-out,taxi-in,arrival-time,arrival-delay,air-time,carrier-delay,weather-delay,nas-delay,security-delay,late-aircraft-delay,month,day-of-the-month,day-of-the-week,delayed,status
342111,2019-02-12,163,185,0.808670,0.016538,0.13750,0.064516,0.888704,0.037459,0.269400,0.0,0.0,0.019608,0.0,0.000000,2,12,1,True,1
298420,2019-07-14,57,122,0.885369,0.018008,0.05000,0.048387,0.977491,0.020198,0.162518,0.0,0.0,0.016340,0.0,0.003692,7,14,6,False,0
275467,2019-04-27,243,20,0.431013,0.017273,0.04375,0.008065,0.556065,0.020198,0.147877,0.0,0.0,0.013072,0.0,0.007383,4,27,5,False,0
271709,2019-06-03,163,187,0.596082,0.015803,0.11875,0.024194,0.648604,0.024238,0.250366,0.0,0.0,0.009804,0.0,0.011075,6,3,0,False,0
50814,2019-10-09,232,207,0.841601,0.016538,0.06250,0.000000,0.931638,0.017260,0.081991,0.0,0.0,0.006536,0.0,0.014766,10,9,2,False,0


In [11]:
# save the data frame obtained after all the feature engineering tasks
toHdf5FeatureEng(df_encoded, path)